In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os

import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [ ]:
#Importación de archivos de afiliados
inicio = time.time()
afi = pd.read_csv( ruta_afi, delimiter='\t', encoding='iso-8859-1', decimal='.')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

In [ ]:
#Agrupamiento por relación de trabajo
inicio = time.time()
agrupa = [ (afi['SECTOR'].isin(['PRIVADO', 'PASANTE PRIVADO'])), 
           (afi['SECTOR'].isin(['PUBLICO', 'PASANTE PUBLICO'])), 
           (afi['SECTOR'].isin(['INDEPENDIENTES', '108-ARTISTA Y GESTOR DE CULTURA', 
                                '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  '])),
           (afi['SECTOR'].isin(['69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
                                '89-AFILIACION DOMICILIADO EN EL EXTERIOR'])),
           (afi['SECTOR'] == '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA'),
           (afi['SECTOR'] == '06-CODIGO DEL TRABAJO - CT ')
            ]
            
tipo = ['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR']

afi['SECTOR_A'] = np.select( agrupa, tipo, default=afi['SECTOR'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

In [ ]:
afi['SECTOR'].unique()

In [ ]:
afi['SECTOR_A'].unique()

In [ ]:
afi.shape

In [ ]:
#Para acceder a las prestaciones de IVM debe tener al menos 5 años de aporte o 60 imposiciones
ced_counts = afi.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()

In [ ]:
# numero de cedulas que tienen menos de 60 imposiciones
ced_counts[ced_counts < 60].index.nunique()

In [ ]:
#Se filtran a las cedulas con menos de 60 imposicones 
inicio = time.time()
afi_fil = afi[~afi['CEDULA_COD'].isin(ced_counts[ced_counts < 60].index)].copy()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se crea una variable fecha
inicio = time.time()
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se ordenan los registros
inicio = time.time()
afi_fil = afi_fil.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
afi_fil.shape

In [ ]:
# Especifica la ruta del directorio donde quieres guardar el archivo
directorio = r_ruta

# Asegúrate de que el directorio existe
os.makedirs(directorio, exist_ok=True)

# Nombre del archivo
nombre_archivo = 'viu_clean_afiliados.pkl'

# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)

# Objetos a guardar
objeto1 = afi
objeto2 = afi_fil

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto1, archivo)
    pickle.dump(objeto2, archivo)
    
# #Cargar los archivos
# with open( ruta_archivo, 'rb') as archivo:
#     pickle.load(archivo)  # Carga y descarta el primer objeto
#     afi_fil = pickle.load(archivo)  # Carga el segundo objeto    

In [ ]:
#Se eliminan archivos no necesarios para liberar la memoria
del afi
del objeto1
del objeto2
del ced_counts

In [ ]:
afi_fil #Tiene una dimensión de (64638969 rows × 14 columns)

In [ ]:
#Se seleccionan las características de la población para los año y mes de aportación
afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES', 'FECHA','SALARIO', 'SECTOR_A']].copy()
del afi_fil

In [ ]:
#Inicio de la HL
caract = afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index() 
caract.rename( columns={'FECHA': 'INI_HL'}, inplace=True)
#Fin de la HL
caract['FIN_HL'] = caract['CEDULA_COD'].map( afi_sel.groupby('CEDULA_COD')['FECHA'].max() )
#Contar los meses, considerando duplicidad en los meses que puede tener aporte simultaneos
caract['MES_AS'] = caract['CEDULA_COD'].map( afi_sel['CEDULA_COD'].value_counts() )
#Meses trabajados unicos
aux = afi_sel.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])
caract['MES_TU'] = caract['CEDULA_COD'].map( aux['CEDULA_COD'].value_counts() )

In [ ]:
objeto3 = caract

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto3, archivo)

In [ ]:
del objeto3

In [ ]:
caract

In [ ]:
#Personas que tienen varios aportes simultaneos
caract[caract['MES_AS']>caract['MES_TU']]

In [ ]:
#afi_sel[afi_sel['CEDULA_COD']==21729781].tail(60)

In [ ]:
afi_sel =  afi_sel.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel['NUM_SEC_MES'] = 1
afi_sel['%_NUM_SECTOR'] = afi_sel['SALARIO']

In [ ]:
aux = afi_sel[afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]
aux1 = afi_sel[~afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]

In [ ]:
#Se debe analizar a las personas que tienen varios aportes en un mismo mes
#Se suman los salarios y se concatenan los sectores para la cedula que en el mismo anio y mes tienen más de un sector
inicio = time.time()
afi_sel_g = aux.groupby(['CEDULA_COD', 'ANIO', 'MES']).agg({'SALARIO': 'sum',
                                                            'SECTOR_A': lambda x: ';'.join(x),
                                                            'NUM_SEC_MES': lambda x: len(x),
                                                            '%_NUM_SECTOR': lambda x:  ';'.join((x / x.sum()).round(4).astype(str))
                                                                }).reset_index()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


In [ ]:
afi_sel_g

In [ ]:
afi_sel_g_all = pd.concat( [aux1[['CEDULA_COD', 'ANIO', 'MES', 'SALARIO', 'SECTOR_A', 'NUM_SEC_MES', '%_NUM_SECTOR']], 
                            afi_sel_g], ignore_index=True )

In [ ]:
afi_sel_g_all

In [ ]:
objeto4 = afi_sel_g_all

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto4, archivo)

In [4]:
#afi_sel_g_all = objetos[3]

In [ ]:
del aux
del aux1
del objeto4

In [ ]:
afi_sel_g_all = afi_sel_g_all.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel_g_all.loc[ (afi_sel_g_all['NUM_SEC_MES'] == 1), '%_NUM_SECTOR'] = '1'

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==216]

In [ ]:
#Se reinician los index
afi_sel_g_all.reset_index(inplace=True)
afi_sel_g_all.rename(columns={'index': 'nuevo_indice'}, inplace=True)
afi_sel_g_all.drop(columns=['nuevo_indice'], inplace=True)

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126]

In [ ]:
#se crean los grupos de 12 meses trabajados para tener
n_grupo = 12
afi_sel_g_all['GRUPO'] = (afi_sel_g_all.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
#Se saca el salario promedio de cada grupo
afi_sel_g_all['SAL_PROM_GRUPO'] = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')

In [ ]:
afi_sel_g_all

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126]['GRUPO'].unique()

In [ ]:
#Se obtienen los sueldos promedios para cada grupo y cada cedula
inicio = time.time()
df1 = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].first().reset_index().copy()
df1 = df1.sort_values(by = ['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
df1

In [ ]:
 df1[df1['CEDULA_COD']==126]

In [ ]:
#Se seleccionan los 5 mejores años de sueldo
inicio = time.time()
top = 5
#top_sal = df1.groupby('CEDULA_COD')['SAL_PROM_GRUPO'].nlargest(top).reset_index(level=0, drop=True)
indices = df1.groupby('CEDULA_COD').apply(lambda x: x.index[:top]).explode()
df2 = df1.loc[indices].reset_index(drop=True)
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
df2[df2['CEDULA_COD']==126]

In [ ]:
#Seleccionar los cinco mejores años del grupo total de años de aporte
combinaciones = set(zip(df2['CEDULA_COD'], df2['GRUPO']))
afi_sel_g_all['GRUPO_SEL'] = [(cedula, grupo) in combinaciones for cedula, grupo in zip(afi_sel_g_all['CEDULA_COD'], afi_sel_g_all['GRUPO'])]
afi_sel_g_all['GRUPO_SEL'] = afi_sel_g_all['GRUPO_SEL'].astype(int)

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126].tail(60)

In [ ]:
#Se crea una variable fecha para el período de mejores años y la base de cálculo
inicio = time.time()
mejores = afi_sel_g_all[afi_sel_g_all['GRUPO_SEL']==1].copy()
mejores.loc[:, 'FECHA'] = pd.to_datetime(mejores['ANIO'].astype(str) + '-' + mejores['MES'].astype(str).str.zfill(2) + '-01')

#Se calcula la base de calculo
df_prom = mejores.groupby('CEDULA_COD')['SALARIO'].mean()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
mejores

In [ ]:
afi_sel_g_all['INI_CAL'] = afi_sel_g_all['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].max() )
afi_sel_g_all['FIN_CAL'] = afi_sel_g_all['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].min() )
afi_sel_g_all['BASE_CAL'] = afi_sel_g_all['CEDULA_COD'].map( df_prom )

In [ ]:
caract['INI_CAL'] = caract['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].max() )
caract['FIN_CAL'] = caract['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].min() )
caract['BASE_CAL'] = caract['CEDULA_COD'].map( df_prom )
caract

In [ ]:
del df1, df2, mejores

In [ ]:
afi_sel_g_all[ (afi_sel_g_all['CEDULA_COD']==126) & (afi_sel_g_all['GRUPO_SEL']==1)]['SALARIO'].mean()

In [ ]:
afi_sel_g_all[ (afi_sel_g_all['CEDULA_COD']==126)]

In [ ]:
# #     df = asignar_grupo(sect, n_grupo)
# #     df1 = valores_unicos_grupos(df)
# #     df2 = sel_top_salarios(df1, top)
# #     df = sel_grupo(df, df2)
#     df = fec_sel_grupo(df)
#     df = base_cal(df)
#     df = ati_base_cal_m1(df, inferior)
#     df = base_cal_sin_ati_m1(df)
#     df = sbu_ajuste_m1(df, SBU)

In [ ]:
#Registros con años menores al 2000
afi_sel_g_all[afi_sel_g_all['FIN_CAL'].dt.year<2000]

In [ ]:
#Se agrega el valor del SBU
SBU = pd.DataFrame( { 'ANIO':[2000,2001,2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
                              2021, 2022, 2023, 2024],
                      'VALOR':[57,85.65,105, 122, 136, 150, 160, 170, 200, 218, 240, 264, 292, 318, 340, 354, 366, 375, 386, 394, 400, 
                              400, 425, 450, 460]})
SBU

In [ ]:
afi_sel_g_all['SBU'] = afi_sel_g_all['ANIO'].map( SBU.groupby('ANIO')['VALOR'].first())

In [ ]:
afi_sel_g_all

In [ ]:
#Cédula que tienen su historia laboral igual al SBU
afi_sel_g_all['ID_SBU'] = afi_sel_g_all['SALARIO'] == afi_sel_g_all['SBU']
#Si tiene -1 es un valor que no es un SBU en ese año y mes
afi_sel_g_all['ID_SBU'] = afi_sel_g_all['ID_SBU'].astype(int)-1
afi_sel_g_all

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126].shape

In [ ]:
afi_sel_g_all[(afi_sel_g_all['CEDULA_COD']==126) & (afi_sel_g_all['GRUPO_SEL']==1)]

In [ ]:
objeto5 = afi_sel_g_all

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto5, archivo)

In [ ]:
del objeto5

In [ ]:
#Selección de cedulas con sus mejores años de sueldo y que su salario no sea un SBU
data_sin_ati = afi_sel_g_all[ (afi_sel_g_all['GRUPO_SEL']==1) & (afi_sel_g_all['ID_SBU']!=-1)]
data_ati = afi_sel_g_all[ (afi_sel_g_all['GRUPO_SEL']==1) & (afi_sel_g_all['ID_SBU']!=0)]

In [ ]:
data_ati.shape #(21022610, 15)
data_sin_ati.shape #(5449506, 15)

In [ ]:
data_ati[data_ati['CEDULA_COD']==126][['CEDULA_COD','ANIO','MES','SALARIO']].shape

In [ ]:
data_ati[data_ati['CEDULA_COD']==126]

In [ ]:
objeto6 = data_ati
objeto7 = data_sin_ati

# Guardar los objetos en el archivo
with open(ruta_archivo, 'ab') as archivo: 
    pickle.dump(objeto6, archivo)
    pickle.dump(objeto7, archivo)
    
del objeto6, objeto7

In [3]:
#Cargar los archivos
directorio = r_ruta
nombre_archivo = 'viu_clean_afiliados.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

objetos = []
try:
    with open(ruta_archivo, 'rb') as archivo:
        while True:
            try:
                objetos.append(pickle.load(archivo))
            except EOFError:
                break
except Exception as e:
    print(f"Ocurrió un error al intentar cargar el archivo: {e}")

# Verifica la cantidad de objetos y accede a ellos
print(f"El archivo contiene {len(objetos)} objetos.")
for i, obj in enumerate(objetos):
    print(f"Objeto {i+1}: {obj}")

# # Acceder a un objeto específico
# if len(objetos) >= 3:
#     afi_sel_g_all = objetos[2]  # Cargar el tercer objeto
# else:
#     print("El archivo no contiene suficientes objetos.")

El archivo contiene 6 objetos.
Objeto 1:           CEDULA_COD  ANIO  MES CODSEC          SECTOR  \
0           16260000  2003   12      R         PRIVADO   
1           10788805  1994    4      R         PRIVADO   
2            1866110  1991   12      R         PRIVADO   
3            3579340  1986    5      R         PRIVADO   
4           10605673  2003   11      R         PRIVADO   
...              ...   ...  ...    ...             ...   
68062265    13322434  2017    9      R         PRIVADO   
68062266    10495276  2017   11      R         PRIVADO   
68062267     1503916  2019   12      P         PUBLICO   
68062268    16254330  2019    8      R  INDEPENDIENTES   
68062269     6091299  2020    7      P         PUBLICO   

                                                  DESTIPEMP  \
0                 2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS   
1                3-EMPRESA UNIPERSONAL /  PEQUEÑA INDUSTRIA   
2                                    25-EMPLEADOR DOMESTICO   
3         

In [ ]:
#Análisis de atípicos
#Método de clustering jerárquico
data_ati = objetos[4]
data_sin_ati = objetos[5]

In [ ]:
#data_ati = data_ati[ data_ati['SALARIO'] >= data_ati['SBU']]
data_ati = data_ati.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
data_ati

In [ ]:
data_ati[data_ati['CEDULA_COD']==1637]
#data_sin_ati[data_sin_ati['CEDULA_COD']==8748]

In [ ]:
data_ati_dic = data_ati.groupby('CEDULA_COD')['SALARIO'].apply(list).to_dict()
data_ati_dic

In [ ]:
#cedula
data_ati_dic[126]

In [ ]:
#Algoritmo jerarquico basado en las derivadas
data_val_ati = {}
for cedula in data_ati_dic:
    if( len( np.unique(data_ati_dic[cedula] ) ) > 3 ):
        aux = np.array( np.unique( data_ati_dic[cedula] ) ).reshape(-1, 1)
        Z = linkage( aux , method='single', metric='euclidean')
        last = Z[-10:, 2]
        last_rev = last[::-1]
        idxs = np.arange(1, len(last) + 1)
        acceleration = np.diff(last, 2)  # Segunda derivada de las distancias
        acceleration_rev = acceleration[::-1]

        k = acceleration_rev.argmax() + 2  # El codo en la gráfica de la segunda derivada
        max_d = last_rev[k]

        # Cortar el dendrograma a la altura determinada
        clusters = fcluster(Z, max_d, criterion='distance')
        unique, counts = np.unique(clusters, return_counts=True)
        atypical_clusters = unique[counts == 1]
        atypical_values = np.isin(clusters, atypical_clusters).astype(int)
    
        data_val_ati[cedula] = {
        'SALARIO': aux.flatten().tolist(),
        'ATIPICO':  atypical_values.tolist()
        }
    else:
        data_val_ati[cedula] = {
        'SALARIO': aux.flatten().tolist(),
        'ATIPICO':  -1
        }


In [ ]:
cedula

In [ ]:
data_val_ati

In [ ]:
aux=None
aux = np.array( data_ati_dic[126] ).reshape(-1, 1)
print(aux)
Z = linkage( aux , method='single', metric='euclidean')
dendrograma = dendrogram(Z, no_plot=False)

# Realizar el corte del dendrograma para obtener los clusters
num_clusters = 2  # Puedes ajustar este valor según tus necesidades
clusters = fcluster(Z, num_clusters, criterion='maxclust')

# Calcular el centroide de cada clúster
cluster_centers = np.array([aux[clusters == i].mean(axis=0) for i in range(1, num_clusters + 1)])

# Calcular la distancia de cada punto al centroide de su clúster
distances_to_center = cdist(aux, cluster_centers)

# Calcular las distancias medias a los centroides
mean_distances = distances_to_center.mean(axis=1)

# Calcular un umbral para identificar valores atípicos (por ejemplo, el percentil 95)
threshold = np.percentile(mean_distances, 95)

# Identificar los valores atípicos
outliers_indices = np.where(mean_distances > threshold)[0]
outliers = aux[outliers_indices]

In [ ]:
outliers

In [ ]:
max_d

In [ ]:
60*0.9

In [ ]:
np.diff(last, 2) 